In [ ]:
Real Time Detection Model

In [ ]:
import cv2
import json
import torch

model = torch.load('best.pt')

# Initialize webcam
cap = cv2.VideoCapture(0)

print('Width :' + str(cap.get(3)))
print('Height :' + str(cap.get(4)))

    cap.set(3, 620)
    cap.set(4, 720)

# Real-time detection loop
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Perform detection on the frame
    results = model(frame)

    # Extract predictions and format for display and output
    predictions = []
    group_id_counter = 1

    for pred in results[0].boxes:
        bbox = pred.xyxy[0].cpu().numpy().tolist()

        group_id = getattr(pred, 'group_id', None)
        if group_id is None:
            group_id = group_id_counter
            group_id_counter += 1

        # Append to predictions
        predictions.append({
            "label": "shampoo",
            "points": [
                [bbox[0], bbox[1]],
                [bbox[2], bbox[3]]
            ],
            "group_id": group_id,
            "description": "",
            "shape_type": "rectangle",
            "flags": {}
        })

        # Draw bounding boxes on the frame
        cv2.rectangle(
            frame,
            (int(bbox[0]), int(bbox[1])),
            (int(bbox[2]), int(bbox[3])),
            (255, 0, 0), 2
        )
        cv2.putText(frame, "shampoo", (int(bbox[0]), int(bbox[1]) - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2)

    # Print real-time JSON predictions
    json_output = json.dumps(predictions, indent=4)
    with open("predictions.json", "a") as json_file:
      # If the file is empty, just write the JSON output
      if os.stat("predictions.json").st_size == 0:
          json_file.write(json_output)
      else:
          # If the file is not empty, add a comma before appending
          json_file.write(",\n" + json_output)

    # Display the live video feed with detections
    cv2.imshow('Real-Time Detection', frame)

    # Break the loop if 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release resources
cap.release()
cv2.destroyAllWindows()